In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf

import sys
sys.path.append('/cluster/home/kheuto01/code/covid19-forecasting/tf_model_1p5/')


from model_config import ModelConfig
import os
import subprocess
import pandas as pd
import numpy as np

2022-10-15 16:23:16.291926: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-15 16:23:16.291952: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"}

just_mass_state = {
    
    "Massachusetts": "MA",
}

just_cal_state = {
    
    "California": "CA",
}

all_but_mass  = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"}

all_but_cal = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"}




In [4]:
for learning_rate in [0.1, 0.01]:
    model_config_path = '/cluster/tufts/hugheslab/kheuto01/covid/logs/fix_eff_omicron/100_samples_0.01/MA/final_config.json'
    data_dir = '/cluster/home/kheuto01/code/covid19-forecasting/tf_model_1p5/data/'

    

    for state, state_abbrev in all_but_cal.items():

        #if len(state.split(' ')) ==  2:
        #    continue
        
        rescale_state = 'Massachusetts'
        rescale_state_abbrev = 'MA'
        

        log_dir = f'/cluster/tufts/hugheslab/kheuto01/covid/logs/omicron_new_loss_long/100_samples_{learning_rate}/{state_abbrev}/'

        if not os.path.exists(log_dir):
            
            os.makedirs(log_dir)

        arg_parts = [
                    f"--log_dir {log_dir}",
            f"--learning_rate {learning_rate}",
            f"--data_dir {data_dir}",
            f"--model_config_path {model_config_path}",
                    "--fix_variance",
            f"--state_abbrev {state_abbrev}"]
        
        if state_abbrev != 'MA':
            arg_parts += [
            f"--rescale_state {rescale_state}",
            f"--rescale_state_abbrev {rescale_state_abbrev}",
                ]
        arg_cmd = ' '.join(arg_parts)
        command = (f"args='{arg_cmd}' sbatch < /cluster/home/kheuto01/code/covid19-forecasting/tf_model_1p5/run_experiment.slurm")
        subprocess.run(command, shell=True, check=True)


Submitted batch job 40848461
Submitted batch job 40848462
Submitted batch job 40848463
Submitted batch job 40848464
Submitted batch job 40848465
Submitted batch job 40848466
Submitted batch job 40848467
Submitted batch job 40848468
Submitted batch job 40848469
Submitted batch job 40848470
Submitted batch job 40848471
Submitted batch job 40848472
Submitted batch job 40848473
Submitted batch job 40848474
Submitted batch job 40848475
Submitted batch job 40848476
Submitted batch job 40848477
Submitted batch job 40848478
Submitted batch job 40848479
Submitted batch job 40848480
Submitted batch job 40848481
Submitted batch job 40848482
Submitted batch job 40848483
Submitted batch job 40848484
Submitted batch job 40848485
Submitted batch job 40848486
Submitted batch job 40848487
Submitted batch job 40848488
Submitted batch job 40848489
Submitted batch job 40848490
Submitted batch job 40848491
Submitted batch job 40848492
Submitted batch job 40848493
Submitted batch job 40848494
Submitted batc

In [4]:
!squeue -u kheuto01 | wc -l

102


#### ModelConfig.from_json('/cluster/tufts/hugheslab/kheuto01/covid/logs/from_config/0.1/saved_config.json')

In [6]:
state

'Wyoming'

In [ ]:
rh